# <center> Deep Learning Project:

> - __Ahmed Abdelazeem__ (m20210433)
> - __Omar Jarir__ (m20201378)  
> - __Chung-Ting Huang__ (m20210437) 

- The objective of this project is to forecast the evolutions of "bitcoin" prices, by using reccurent neural networks.
such as RNN and LSTM.
- The project is organized as follows:

In [ ]:
# !pip install keras-tuner --upgrade
# !pip install -q -U keras-tuner

In [ ]:
import time
t1 = time.perf_counter()

- __Importing the necessary libraries 📚:__

In [ ]:
import os
from zipfile import ZipFile
import numpy as np
import random as python_random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm 
import tensorflow as tf
from tensorflow import keras 
import keras_tuner as kt

In [ ]:
from keras.models import Model, Sequential
from keras.layers import Dense, BatchNormalization, LSTM, Embedding, Input, Dropout, SimpleRNN, RNN, Bidirectional
#from keras.Optimizer import Adam, SGD, RMSprop, Adam, Adadelta, Adagrad, Adamax, Nadam, Ftrl, schedules
from keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger, History, ReduceLROnPlateau 
#from keras.utils import plot_model
from keras import optimizers
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor 
import keras.backend as K

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split, cross_val_score, StratifiedKFold, KFold
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.utils import class_weight

from pprint import pprint

In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn import linear_model
from sklearn import metrics
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
import category_encoders as ce
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,mean_squared_error, ConfusionMatrixDisplay
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from imblearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler
from yellowbrick.classifier import ClassificationReport

import scikitplot as skplt
import xgboost

from mlxtend.evaluate import mcnemar
from mlxtend.evaluate import mcnemar_table

from imblearn.over_sampling import SMOTE

from yellowbrick.model_selection import RFECV
from yellowbrick.model_selection import LearningCurve

from sklearn.ensemble import RandomForestClassifier
import kerastuner as kt

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
SEED = 2022

LEARNING_RATE = 0.001
BATCH_SIZE = 256
EPOCHS = 10
time_steps=5

Fixing the random number seed to ensure our results are reproducible.

In [ ]:
session = K.get_session()
# init_op = tf.group(tf.tables_initializer(),tf.global_variables_initializer(),
# tf.local_variables_initializer())
#session.run(init_op)

np.random.seed(SEED)
tf.random.set_seed(SEED)
python_random.seed(SEED)

- __Helper functions:__

In [ ]:
# Function to create dataframe with metrics

def performanceMetricsDF(metricsObj, yTrain, yPredTrain, yTest, yPredTest):
    measures_list = ['ACCURACY','PRECISION', 'RECALL','F1 SCORE','AUC']
    train_results = [metricsObj.accuracy_score(yTrain, yPredTrain),
                metricsObj.precision_score(yTrain, yPredTrain),
                metricsObj.recall_score(yTrain, yPredTrain, average='weighted'),
                metricsObj.f1_score(yTrain, yPredTrain, average='weighted'),
                metricsObj.roc_auc_score(yTrain, yPredTrain),    
                ]
    test_results = [metricsObj.accuracy_score(yTest, yPredTest),
               metricsObj.precision_score(yTest, yPredTest),
               metricsObj.recall_score(yTest, yPredTest, average='weighted'),
               metricsObj.f1_score(yTest, yPredTest, average='weighted'),
               metricsObj.roc_auc_score(yTest, yPredTest), 
               ]
    resultsDF = pd.DataFrame({'Measure': measures_list, 'Train': train_results, 'Test':test_results})
    return(resultsDF)

In [ ]:
# Function to plot confusion matrix - Adapted from https://github.com/DTrimarchi10/confusion_matrix/blob/master/cf_matrix.py
def make_confusion_matrix(cf,
                          group_names=None,
                          categories='auto',
                          count=True,
                          percent=True,
                          cbar=True,
                          xyticks=True,
                          xyplotlabels=True,
                          sum_stats=True,
                          figsize=None,
                          cmap='Blues',
                          title=None):
    '''
    This function will make a pretty plot of an sklearn Confusion Matrix cm using a Seaborn heatmap visualization.
    Arguments
    ---------
    cf:            confusion matrix to be passed in
    group_names:   List of strings that represent the labels row by row to be shown in each square.
    categories:    List of strings containing the categories to be displayed on the x,y axis. Default is 'auto'
    count:         If True, show the raw number in the confusion matrix. Default is True.
    normalize:     If True, show the proportions for each category. Default is True.
    cbar:          If True, show the color bar. The cbar values are based off the values in the confusion matrix.
                   Default is True.
    xyticks:       If True, show x and y ticks. Default is True.
    xyplotlabels:  If True, show 'True Label' and 'Predicted Label' on the figure. Default is True.
    sum_stats:     If True, display summary statistics below the figure. Default is True.
    figsize:       Tuple representing the figure size. Default will be the matplotlib rcParams value.
    cmap:          Colormap of the values displayed from matplotlib.pyplot.cm. Default is 'Blues'
                   See http://matplotlib.org/examples/color/colormaps_reference.html
                   
    title:         Title for the heatmap. Default is None.
    '''


    # CODE TO GENERATE TEXT INSIDE EACH SQUARE
    blanks = ['' for i in range(cf.size)]

    if group_names and len(group_names)==cf.size:
        group_labels = ["{}\n".format(value) for value in group_names]
    else:
        group_labels = blanks

    if count:
        group_counts = ["{0:0.0f}\n".format(value) for value in cf.flatten()]
    else:
        group_counts = blanks

    if percent:
        group_percentages = ["{0:.2%}".format(value) for value in cf.flatten()/np.sum(cf)]
    else:
        group_percentages = blanks

    box_labels = [f"{v1}{v2}{v3}".strip() for v1, v2, v3 in zip(group_labels,group_counts,group_percentages)]
    box_labels = np.asarray(box_labels).reshape(cf.shape[0],cf.shape[1])


    # CODE TO GENERATE SUMMARY STATISTICS & TEXT FOR SUMMARY STATS
    if sum_stats:
        #Accuracy is sum of diagonal divided by total observations
        accuracy  = np.trace(cf) / float(np.sum(cf))

        #if it is a binary confusion matrix, show some more stats
        if len(cf)==2:
            #Metrics for Binary Confusion Matrices
            precision = cf[1,1] / sum(cf[:,1])
            recall    = cf[1,1] / sum(cf[1,:])
            f1_score  = 2*precision*recall / (precision + recall)
            stats_text = "\n\nAccuracy={:0.3f}\nPrecision={:0.3f}\nRecall={:0.3f}\nF1 Score={:0.3f}".format(
                accuracy,precision,recall,f1_score)
        else:
            stats_text = "\n\nAccuracy={:0.3f}".format(accuracy)
    else:
        stats_text = ""


    # SET FIGURE PARAMETERS ACCORDING TO OTHER ARGUMENTS
    if figsize==None:
        #Get default figure size if not set
        figsize = plt.rcParams.get('figure.figsize')

    if xyticks==False:
        #Do not show categories if xyticks is False
        categories=False


    # MAKE THE HEATMAP VISUALIZATION
    plt.figure(figsize=figsize)
    ax = sns.heatmap(cf,annot=box_labels, fmt="",cmap=cmap,cbar=cbar,xticklabels=categories,yticklabels=categories)

    if xyplotlabels:
        plt.ylabel('True label')
        plt.xlabel('Predicted label' + stats_text)
    else:
        plt.xlabel(stats_text)
    
    if title:
        plt.title(title)

In [ ]:
# Function to find outliers on columns based on percentile
def removeOutliers(df, colList, lowPercentile=0.05, highPercentile=0.95, verbose=False):
    quant_df = df[colList].quantile([lowPercentile, highPercentile])
    if verbose:
        print(quant_df)
    for name in list(df[colList].columns):
        df = df[(df[name] >= quant_df.loc[lowPercentile, name]) & (df[name] <= quant_df.loc[highPercentile, name])]
    return df

In [ ]:
def create_model(nl=1, nn=256):
    model = Sequential()
    model.add(Dense(16, input_shape=(2,), activation="relu"))
    # Add as many hidden layers as specified in nl
    for i in range(nl):
        # Layers have nn neurons
        model.add(Dense(nn, activation='relu'))
    # End defining and compiling your model...

In [ ]:
def show_raw_visualization(data):
    time_data = data[date_time_key]
    fig, axes = plt.subplots(
        nrows=7, ncols=2, figsize=(15, 20), dpi=80, facecolor="w", edgecolor="k"
    )
    for i in range(len(feature_keys)):
        key = feature_keys[i]
        c = colors[i % (len(colors))]
        t_data = data[key]
        t_data.index = time_data
        t_data.head()
        ax = t_data.plot(
            ax=axes[i // 2, i % 2],
            color=c,
            title="{} - {}".format(titles[i], key),
            rot=25,
        )
        ax.legend([titles[i]])
    plt.tight_layout()
    plt.show()

In [ ]:
def show_heatmap(data):
    plt.matshow(data.corr())
    plt.xticks(range(data.shape[1]), data.columns, fontsize=14, rotation=90)
    plt.gca().xaxis.tick_bottom()
    plt.yticks(range(data.shape[1]), data.columns, fontsize=14)

    cb = plt.colorbar()
    cb.ax.tick_params(labelsize=14)
    plt.title("Feature Correlation Heatmap", fontsize=14)
    plt.tight_layout()
    plt.show()


In [ ]:
# This function plot the "loss" and the "val_loss" variables.
def visualize_loss(history, title):
    """
    history: history of the model.
    title: Title of the plot.
    """
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]
    epochs = range(1, len(loss)+1)
    plt.plot(epochs, loss, "g", label="Training loss")
    plt.plot(epochs, val_loss, "b", label="Validation loss")
    plt.title(title)
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
# This function plot the "accuracy" and the "val_accuracy" variables.
def visualize_accuracy(history, title):
    """
    history: history of the model.
    title: Title of the plot.
    """
    loss = history.history["accuracy"]
    val_loss = history.history["val_accuracy"]
    epochs = range(1, len(loss)+1)
    plt.plot(epochs, loss, "g", label="Training accuracy")
    plt.plot(epochs, val_loss, "b", label="Validation accuracy")
    plt.title(title)
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
# This function plot the "error" and the "val_error" variables.
def visualize_error(history, title):
    """
    history: history of the model.
    title: Title of the plot.
    """
    error = history.history["mae"]
    val_error = history.history["val_mae"]
    epochs = range(len(error))
    plt.figure(figsize=(10, 10))
    plt.plot(epochs, error, "b", label="Training error")
    plt.plot(epochs, val_error, "r", label="Validation error")
    plt.title(title)
    plt.xlabel("Epochs")
    plt.ylabel("Error")
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
def show_plot(plot_data, delta, title):
    """
    plot_data:
    delta:
    title:
    """
    labels = ["History", "True Future", "Model Prediction"]
    marker = [".-", "rx", "go"]
    time_steps = list(range(-(plot_data[0].shape[0]), 0))
    if delta:
        future = delta
    else:
        future = 0

    plt.title(title)
    for i, val in enumerate(plot_data):
        if i:
            plt.plot(future, plot_data[i], marker[i], markersize=10, label=labels[i])
        else:
            plt.plot(time_steps, plot_data[i].flatten(), marker[i], label=labels[i])
    plt.legend()
    plt.xlim([time_steps[0], (future + 5) * 2])
    plt.xlabel("Time-Step")
    plt.tight_layout()
    plt.show()
    return

In [ ]:
def plot_predictions(val, predicted, title, date):
    plt.figure(figsize=(16,4))
    plt.plot(date, val, color='blue',label='Actual') # date, 
    plt.plot(date,predicted, alpha=0.7, color='red',label='Predict') # date, 
    plt.title(title)
    plt.xlabel('Date')
    plt.ylabel('Something')
    plt.legend()
    plt.tight_layout()
    plt.show()

- __Loading the dataset:__

In [ ]:
ds= pd.read_csv("data.csv")

In [ ]:
ds.head()

In [ ]:
# Checking the shape of the dataset:

ds.shape

In [ ]:
# This is the target variable.
ds["Bankrupt?"].unique()

In [ ]:
# Initializing the model:

rf_model = RandomForestClassifier(n_estimators=10, random_state=SEED)

In [ ]:
%%timeit

# I get to select 17 features.

# Applying RFECV (Recursive Feature Elimination) to select features
# see https://www.scikit-yb.org/en/latest/api/model_selection/rfecv.html
cv = StratifiedKFold(3)
vis = RFECV(rf_model, scoring='f1_weighted') ## , cv=cv, we dont need to use cross validation
vis.fit(ds.drop(columns="Bankrupt?"), ds["Bankrupt?"])
vis.show()

In [ ]:
vis.ranking_

In [ ]:
# Features to include: 
ds.columns[vis.support_]

In [ ]:
# This should be the the new data set.

ds[ds.columns[vis_support_]]

In [ ]:
ds

In [ ]:
# Checking that the target distribution is imbalanced.

sns.countplot(y='Bankrupt?', data = ds, palette='viridis', orient = 'h')


# Data Exploration:

In [ ]:
ds.head()

In [ ]:
# We can see that our dataset does not contain any duplicates.

ds.duplicated().sum()

In [ ]:
# Checking the type of the columns
ds.dtypes

# Data Pre-processing:

In [ ]:
X = ds.copy(deep=True)

## Dealing with outliers:

In [ ]:
X = removeOutliers(X, colList=cols , lowPercentile=0.05, highPercentile=0.95, verbose=True) 

## One Hot encoding the categorical features:

In [ ]:
# Transforming a list of columns to categorical.
# I need to get the categorical features:

cols = ["", "", ""]
ds[cols] = ds[cols].apply(lambda x:x.astype("category"))

In [ ]:
# Encode categorical values into dummy variables.

cols = ["", "", ""]
ce_one_hot = ce.OneHotEncoder(cols = cols, use_cat_names=True)
X = ce_one_hot.fit_transform(X)

In [ ]:
y = X['']
X = X.drop(columns=['Bankrupt?'])

## Splitting the dataset:

In [ ]:
# Split the dataset intro train and test sets.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,
                                   shuffle =True, stratify=y, random_state=SEED)

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

## Data Normalization:

In [ ]:
# We use the standard scaler in order to normalize the data:

sc = StandardScaler() 
X_train = sc.fit_transform(X_train) 
X_test = sc.transform(X_test)

# Modelling:

## Building The First Model:

In [ ]:
METRICS = [
    FalseNegatives(name='fn'), BinaryAccuracy(name='accuracy'),
    Precision(name='precision'), Recall(name='recall'), AUC(name='auc'),]

In [ ]:
# Building the first model

def create_model(nn=16, nl=1, learning_rate=0.001, activation="relu", optim="adam", 
                 init="glorot_uniform", loss = "binary_crossentropy"):
    classifier = Sequential()
    # Let's add the first hidden layers
    classifier.add(Dense(units=nn, kernel_initializer=init, activation=activation,
                         input_dim=X_train.shape[1])) 
    
    for _ in range(nl):
        classifier.add(Dense(units=8, kernel_initializer=init, activation=activation))
        classifier.add(Dense(units=1, kernel_initializer=init, activation="sigmoid"))
    
    classifier.compile(optimizer=optim, loss=loss, metrics=["accuracy"])
    return classifier

### Hyper parameters tuning:

#### Using Keras tuner:

In [ ]:
# This function is similar to the one above:

def get_model_design_hp(hp, nl=1):
    
    classifier = Sequential()
    classifier.add(Dense(units=hp.Int('First Layer',16,32,step=8,default=16),
                              kernel_initializer="glorot_uniform", activation="relu",
                         input_dim=X_train.shape[1])) 
         
    for _ in range(nl):
        classifier.add(Dense(units=hp.Int('First Layer',4,8,step=4,default=4), kernel_initializer="glorot_uniform", 
                             activation="relu"))
        classifier.add(Dense(units=1, kernel_initializer="glorot_uniform", activation="sigmoid"))
            
    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')),
                loss='binary_crossentropy',metrics= METRICS) 
    
    return model

In [ ]:
# Does this use cross validation or not?

tuner = kt.Hyperband(
    get_model_design_hp,
    objective= 'val_loss' ,
    max_epochs=100,
    hyperband_iterations=2,
    directory = LOG_DIR    
)

In [ ]:
tuner.search(X_train, y_train
             validation_data = (X_test, y_test),
             epochs=2,
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=3)])

In [ ]:
best_model = tuner.get_best_models(1)[0]

In [ ]:
best_hyperparameters = tuner.get_best_hyperparameters(1)[0]

#### Using Scikit Learn wrapper:

https://machinelearningmastery.com/use-keras-deep-learning-models-scikit-learn-python/

In [ ]:
keras_classifier = KerasClassifier(build_fn=create_model, verbose=1)
# epochs=50, batch_size=256

In [ ]:
# create model
# evaluate using 10-fold cross validation

#kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=SEED)
#results = cross_val_score(keras_classifier, X_train, y_train, cv=kfold)
#print(results.mean())
#print(results.std())

In [ ]:
optimizers = ['rmsprop', 'adam']
init = ['glorot_uniform', 'normal', 'uniform']
epochs = [50, 100, 150]
batches = [64, 128, 256]

param_grid = dict(optim=optimizers, epochs=epochs, batch_size=batches, init=init)

random_search = RandomizedSearchCV(estimator=keras_classifier, param_distributions=param_grid)

random_search_results = grid.fit(X_train, y_train)


# summarize results
print("Best: %f using %s" % (random_search_results.best_score_, random_search_results.best_params_))

In [ ]:
means = random_search_results.cv_results_['mean_test_score']
stds = random_search_results.cv_results_['std_test_score']
params = random_search_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

#### Building the model:

In [ ]:
# Now that we have the best hyper parameters we can build the model:

# classifier = create_model(nn=16, here i should include all the found parameters)

In [ ]:
classifier.summary()

In [ ]:
# Path + name of the file we wanna create.
path_checkpoint = "First_model.h5"

early_stopping = EarlyStopping(monitor="val_loss", patience=10, mode="auto")

model_save = ModelCheckpoint(monitor="val_loss", filepath=path_checkpoint, verbose=1, 
                             save_best_only=True)

Hist_first = History()

In [ ]:
hist = classifier.fit(X_train, y_train,  
                      batch_size=64,
                      validation_split=0.2,
                      # the number of epochs can be high, early stopping will stop training when the val_loss is not improving
                      epochs=1000, 
                      callbacks = [early_stopping, model_save, Hist_first])

In [ ]:
y_pred_train = classifier.predict(X_train)
y_pred_test = classifier.predict(X_test)

In [ ]:
# Converting the predictions to the right format:

y_pred_train=(y_pred_train>0.5).astype(int)
y_pred_test=(y_pred_test>0.5).astype(int)

In [ ]:
# Showing the confusion matrix
cm = metrics.confusion_matrix(y_test,y_pred_test)
labels = ['True Neg','False Pos','False Neg','True Pos']
categories = ['0', '1']
make_confusion_matrix(cm, group_names=labels, categories=categories, cmap='Blues')

In [ ]:
# Showing the classification report:

target_names=["No Default", "Default"]
print(classification_report(y_test, y_pred_test, target_names=target_names))

In [ ]:
# Showing the results performance.

resultsDF = performanceMetricsDF(metrics, y_train, y_pred_train, 
                                 y_test, y_pred_test)
resultsDF

In [ ]:
# Visualizing the train and validation loss:

visualize_loss(hist, "Training and validation loss")

In [ ]:
# Visualizing the train and validation accuracy:

visualize_accuracy(hist, "Training and validation accuracy")

In [ ]:
# Maybe i can also add other plots.

## Using class_weight:

In [ ]:
# Getting the weights of each class

class_weights=dict(enumerate(class_weight.compute_class_weight('balanced', 
                            classes=np.unique(y_train), y=y_train)))

In [ ]:
# Now that we have the best hyper parameters we can build the model:

# classifier = create_model(nn=16, here i should include all the found parameters)

In [ ]:
# Path + name of the file we wanna create.
path_checkpoint = "Second_model.h5"

early_stopping = EarlyStopping(monitor="val_loss", patience=10, mode="auto")

model_save = ModelCheckpoint(monitor="val_loss", filepath=path_checkpoint, verbose=1, 
                             save_best_only=True)

Hist_second = History()

In [ ]:
# I SHOULD PAY ATTENTION TO THE MODEL NAME.

hist = classifier.fit(X_train, y_train,  
                      batch_size=64,
                      validation_split=0.2,
                      epochs=1000,
                      callbacks = [early_stopping, model_save, Hist_second],  
                      class_weight=class_weights
                     )

In [ ]:
y_pred_train = classifier.predict(X_train)
y_pred_test = classifier.predict(X_test)

In [ ]:
# Converting the predictions to the right format:

y_pred_train=(y_pred_train>0.5).astype(int)
y_pred_test=(y_pred_test>0.5).astype(int)

In [ ]:
# Showing the confusion matrix

cm = metrics.confusion_matrix(y_test,y_pred_test)
labels = ['True Neg','False Pos','False Neg','True Pos']
categories = ['0', '1']
make_confusion_matrix(cm, group_names=labels, categories=categories, cmap='Blues')

In [ ]:
# Showing the classification report:

target_names=["No Default", "Default"]
print(classification_report(y_test, y_pred_test, target_names=target_names))

In [ ]:
# Showing the results performance.

resultsDF = performanceMetricsDF(metrics, y_train, y_pred_train, 
                                 y_test, y_pred_test)
resultsDF

In [ ]:
# Visualizing the train and validation loss:

visualize_loss(hist, "Training and validation loss")

In [ ]:
# Visualizing the train and validation accuracy:

visualize_accuracy(hist, "Training and validation accuracy")

## Using a Focal Loss:

https://github.com/umbertogriffo/focal-loss-keras
- I should add few comments about the focal loss function, and why we choose to use it.
- "focal loss down-weights the well-classified examples. This has the net effect of putting more training emphasis on that data that is hard to classify. In a practical setting where we have a data imbalance, our majority class will quickly become well-classified since we have much more data for it. Thus, in order to insure that we also achieve high accuracy on our minority class, we can use the focal loss to give those minority class examples more relative weight during training."

In [ ]:
def focal_loss(gamma=2., alpha=4.):

    gamma = float(gamma)
    alpha = float(alpha)

    def focal_loss_fixed(y_true, y_pred):
        """Focal loss for multi-classification
        FL(p_t)=-alpha(1-p_t)^{gamma}ln(p_t)
        Notice: y_pred is probability after softmax
        gradient is d(Fl)/d(p_t) not d(Fl)/d(x) as described in paper
        d(Fl)/d(p_t) * [p_t(1-p_t)] = d(Fl)/d(x)
        Focal Loss for Dense Object Detection
        https://arxiv.org/abs/1708.02002

        Arguments:
            y_true {tensor} -- ground truth labels, shape of [batch_size, num_cls]
            y_pred {tensor} -- model's output, shape of [batch_size, num_cls]

        Keyword Arguments:
            gamma {float} -- (default: {2.0})
            alpha {float} -- (default: {4.0})

        Returns:
            [tensor] -- loss.
        """
        epsilon = 1.e-9
        y_true = tf.convert_to_tensor(y_true, tf.float32)
        y_pred = tf.convert_to_tensor(y_pred, tf.float32)

        model_out = tf.add(y_pred, epsilon)
        ce = tf.multiply(y_true, -tf.math.log(model_out))
        weight = tf.multiply(y_true, tf.pow(tf.subtract(1., model_out), gamma))
        fl = tf.multiply(alpha, tf.multiply(weight, ce))
        reduced_fl = tf.reduce_max(fl, axis=1)
        return tf.reduce_mean(reduced_fl)
    return focal_loss_fixed

In [ ]:
# MAKE SURE YOU CHOOSE THE RIGHT PARAMETERS, I NEED TO CHECK THAT OLD FILE AND SEE IF ITS THE SAME OR NOT.

create_model(loss = focal_loss(gamma=1.5,alpha=0.25))

In [ ]:
# Path + name of the file we wanna create.
path_checkpoint = "Third_model.h5"

early_stopping = EarlyStopping(monitor="val_loss", patience=10, mode="auto")

model_save = ModelCheckpoint(monitor="val_loss", filepath=path_checkpoint, verbose=1, 
                             save_best_only=True)

Hist_third = History()

In [ ]:
# I SHOULD PAY ATTENTION TO THE MODEL NAME.

hist = classifier.fit(X_train, y_train,  
                      batch_size=64,
                      validation_split=0.2,
                      epochs=1000,
                      callbacks = [early_stopping, model_save, Hist_third], 
                     )

In [ ]:
y_pred_train = classifier.predict(X_train)
y_pred_test = classifier.predict(X_test)

In [ ]:
# Converting the predictions to the right format:

y_pred_train=(y_pred_train>0.5).astype(int)
y_pred_test=(y_pred_test>0.5).astype(int)

In [ ]:
# Showing the confusion matrix

cm = metrics.confusion_matrix(y_test,y_pred_test)
labels = ['True Neg','False Pos','False Neg','True Pos']
categories = ['0', '1']
make_confusion_matrix(cm, group_names=labels, categories=categories, cmap='Blues')

In [ ]:
# Showing the classification report:

target_names=["No Default", "Default"]
print(classification_report(y_test, y_pred_test, target_names=target_names))

In [ ]:
# Showing the results performance.

resultsDF = performanceMetricsDF(metrics, y_train, y_pred_train, 
                                 y_test, y_pred_test)
resultsDF

In [ ]:
# Visualizing the train and validation loss:

visualize_loss(hist, "Training and validation loss")

In [ ]:
# Visualizing the train and validation accuracy:

visualize_accuracy(hist, "Training and validation accuracy")

## Using S.M.O.T.E:

- Should insert a small introduction about what S.M.O.T.E is.

In [ ]:
# S.M.O.T.E

sm = SMOTE(k_neighbors=10, random_state=SEED)
X_train, y_train = sm.fit_resample(X_train, y_train)

In [ ]:
# MAKE SURE YOU CHOOSE THE RIGHT PARAMETERS, I NEED TO CHECK THAT OLD FILE AND SEE IF ITS THE SAME OR NOT.

# create_model()

In [ ]:
# Path + name of the file we wanna create.
path_checkpoint = "Fourth_model.h5"

early_stopping = EarlyStopping(monitor="val_loss", patience=10, mode="auto")

model_save = ModelCheckpoint(monitor="val_loss", filepath=path_checkpoint, verbose=1, 
                             save_best_only=True)

Hist_fourth = History()

In [ ]:
# I SHOULD PAY ATTENTION TO THE MODEL NAME.

hist = classifier.fit(X_train, y_train,  
                      batch_size=64,
                      validation_split=0.2,
                      epochs=1000,
                      callbacks = [early_stopping, model_save, Hist_third], 
                     )

In [ ]:
y_pred_train = classifier.predict(X_train)
y_pred_test = classifier.predict(X_test)

In [ ]:
# Converting the predictions to the right format:

y_pred_train=(y_pred_train>0.5).astype(int)
y_pred_test=(y_pred_test>0.5).astype(int)

In [ ]:
# Showing the confusion matrix

cm = metrics.confusion_matrix(y_test,y_pred_test)
labels = ['True Neg','False Pos','False Neg','True Pos']
categories = ['0', '1']
make_confusion_matrix(cm, group_names=labels, categories=categories, cmap='Blues')

In [ ]:
# Showing the classification report:

target_names=["No Default", "Default"]
print(classification_report(y_test, y_pred_test, target_names=target_names))

In [ ]:
# Showing the results performance.

resultsDF = performanceMetricsDF(metrics, y_train, y_pred_train, 
                                 y_test, y_pred_test)
resultsDF

In [ ]:
# Visualizing the train and validation loss:

visualize_loss(hist, "Training and validation loss")

In [ ]:
# Visualizing the train and validation accuracy:

visualize_accuracy(hist, "Training and validation accuracy")

## Results discussion:

- Insert results discussion here:

In [ ]:
t2 = time.perf_counter()
print('time taken to run:',(t2-t1)/60.0)